<font color=black size=5 face=雅黑>**GML(Gradual Machine Learning)框架接口文档说明**</font>


用户在使用GML框架进行推理时，需要提供以下三个部分的数据：             
1. 变量          
2. 特征               
3. 简单实例       
     

<font color=black size=4 face=雅黑>**其主要数据结构定义如下:**</font>

In [ ]:
#1.变量(variables)数据结构定义
variables=list() #元素类型为dict:variable
variable =
{
    #由用户提供的属性
    'var_id':1,                  #变量ID   int类型，从0开始
    'is_easy': False,            #是否是Easy
    'is_evidence':True,          #是否是证据变量
    'true_label':1,              #真实标签，用于后续计算精度等
    'label': 1，                 #推理出的标签:0为negative,1为为positive, -1为不知道
    'feature_set':               #此变量拥有的所有feature
    { feature_id1: [theta1,feature_value1],
      feature_id2: [theta2,feature_value2],
      ...
    },
    #代码运行期间可能会自动生成的属性
    'probability':0.99，          #推理出的概率
    'evidential_support':0.3,     #evidential_support
    'entropy': 0.4,               #熵
    'approximate_weight':0.3,     #近似权重
     ...
}

In [ ]:
#2.特征(feature)数据结构定义
features  = list()      #元素类型为dict:feature
feature =
{
    #需要用户提供的属性
    'feature_id':1,                                       #特征id, int类型，从0开始
    'feature_type': unary_feature/binary_feature,         #区分此特征是单因子特征还是双因子特征，目前支持unary_feature和binary_feature两种
    'feature_name': good,                                 #特征名，如果是token类型，就是token具体的词，如果是其他类型的特征，就是特征的具体类型
    'weight':                                             #拥有此特征的所有变量的集合
    {
      var_id1:        [weight_value1,feature_value1],     #unary_feature
     (varid3,varid4): [weight_value2,feature_value2],     #binary_feature
      ...
    }
    #代码运行期间可能会自动生成的属性
    'tau':0,
    'alpha':0,
    'regerssion'： object,
    ...
}


In [ ]:
#3. 简单实例(Easy)数据结构定义 用户也可以直接在variables标明Easy而不提供此文件
easys = list() #元素类型为dict:easy
easy  = 
{
    'var_id': 0,
    'label':  1
}


<font color=black size=4 face=雅黑>**在准备好所需数据之后，主要的调用流程如下：**</font>

In [ ]:
from gml import GML
from easy_instance_labeling import  EasyInstanceLabeling
from evidential_support import EvidentialSupport
from evidence_select import EvidenceSelect
from approximate_probability_estimation import ApproximateProbabilityEstimation

EasyInstanceLabeling(variables,features,easys).label_easy_by_file()
graph = GML(variables, features, evidential_support_method, approximate_probability_method,evidence_select_method, top_m=2000, top_k=10, update_proportion=0.01,
                 balance = False)
graph.inference()            #因子图推理
graph.score()                #获取推理结果

<font color=black size=4 face=雅黑>**目前gml模块中主要的类和函数的详细功能设计如下：**</font>

In [ ]:
#gml类，gml主要流程
Class GML(     
 variables,                         #变量
 features,                           #特征
 evidential_support_method,                  
 #计算evidential support的方法，目前支持两种：regression 和 relation
 approximate_probability_method,
#计算近似概率的方法，目前支持两种：interval 和 relation
 evidence_select_method,
#为待推隐变量挑选证据的方法，目前支持两种：interval 和 relation
 top_m=2000,                                    #topm
 top_k=10,                                          #topk
 update_proportion=0.01,                 #更新比例
 balance = False                                 #是否进行平衡化
 )
#GML类主要函数如下：
1.evidential_support()                                           #计算evidential support
2.approximate_probability_estimation()              #计算近似概率
3.select_top_m_by_es()                                         #选topm
4.select_top_k_by_entropy()                                 #选topk
5.select_evidence()                                               #挑选证据
6.construct_subgraph()                                        #构建子图
7.inference_subgraph()                                        #推理子图
8.label()                                                                #从k个变量中选一个进行标记
9.inference()                                                         #主流程，贯穿以上

In [ ]:
#简单实例标注类
Class EasyInstanceLabeling(variables, features, easys)
#此类目前主要提供以下方法：
1.label_easy_by_file()                          #根据给定的easy文件标easy
2.label_easy_by_clustering()                     #实体识别中标Easy的方法
3.label_easy_by_custom()                         #可由用户自行实现


In [ ]:
#近似概率计算类
Class ApproximateProbabilityEstimation(variables)   
#此类目前主要提供以下方法：
1.approximate_probability_estimation_by_interval()     #实体识别所用方法
2.approximate_probability_estimation_by_relation()    #情感分析所有方法
3.approximate_probability_estimation_by_custom()      #可由用户自行实现


In [ ]:
4.Class EvidentialSupport(			# 计算Evidential Support的类
variables,              					# 变量
features,               					# 特征
method,               					# 计算Evidential Support的方法
evidence_interval_count=10,              	# 区间数
interval_evidence_count=200            	# 每个区间的变量数
)
此类目前主要提供以下方法：
1.evidential_support_by_regression()            
# 计算Evidential Support的方法之一，适用于实体识别
2.evidential_support_by_relation()         
# 计算Evidential Support的方法之一，适用于情感分析
3.evidential_support_by_custom()        	 
# 用户自定义计算Evidential Support的方法



In [ ]:
5.Class EvidenceSelect(				# 挑选证据变量的类
variables,							# 变量
features,							# 特征
interval_evidence_count,				# 每个区间挑选的证据个数
subgraph_limit_num,				# 子图变量上限
k_hop							# 相邻变量的跳数（用于情感分析）
)
此类目前主要提供以下方法：
1.select_evidence_by_interval()			#用于实体识别
2.select_evidence_by_realtion()			#用于情感分析
3.select_evidence_by_custom()			#用户自定义



In [ ]:
6.Class ConstructSubgraph(
variables, 
features,
balance                                                         #是否在建图的时候平衡话
)
此类目前主要提供以下方法：
1.construct_subgraph_for_unaryPara()    #只有单因子（需要参数化），适用于ER
2.construct_subgraph_for_mixture()       #有单双因子(均不需要参数化),适用于ALSA
3.construct_subgraph_for_custom()        #用户自定义


